In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import mahalanobis
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingRegressor
import math
from sklearn.metrics import pairwise_distances

#if this doesn't run then do 'pip install causalinference' in command line
from causalinference import CausalModel

In [2]:
lowDim_dataset = pd.read_csv('../data/lowDim_dataset.csv')
highDim_dataset = pd.read_csv('../data/highDim_dataset.csv')

## A1) Propensity Score Full Matching

### 1. Mahalanobis Metric (Does not need propensity score)

Mahalanobis distance is 
$$D_{ij} = (X_i-X_j)^T\Sigma^{-1}(X_i-X_j)$$
where $\Sigma$ is the covariance matrix of $X$ in the pooled treatment and full control groups.

In [3]:
#testing CausalModel package matching function
#working with the low dimensional data first
cm = CausalModel(
    Y=lowDim_dataset['Y'].values, #response
    D=lowDim_dataset['A'].values, #treatment/control column
    X=lowDim_dataset.iloc[:,2:].values #features
)

In [4]:
cm.est_via_matching()
print(cm.estimates)
#not sure how to interpret this output


Treatment Effect Estimates: Matching

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      2.886      0.412      7.004      0.000      2.079      3.694
           ATC      2.738      0.467      5.862      0.000      1.822      3.654
           ATT      3.366      0.406      8.296      0.000      2.571      4.161



In [5]:
#trying to implement the mahalanobis metric
X=lowDim_dataset.iloc[:,2:].values
A=lowDim_dataset['A'].values
Y=lowDim_dataset['Y'].values

In [6]:
#creating the covariance matrix 
Sigma = np.cov(X)
#not invertible???
#Sigma_inv = np.linalg.inv(Sigma)
np.linalg.det(Sigma)

0.0

The above code isn't needed since it looks like pairwise_distances can find the Mahalanobis matrix for us.

In [7]:
dist_matrix_mahalanobis = pairwise_distances(X,metric='mahalanobis')

### 2. Propensity Score

In [8]:
#creating boosting stump model for calculating propensity score
#not really sure if i'm doing this correctly

#need to do a grid search for best parameters
#we could also do gbm instead of adaboost
adaboost = AdaBoostClassifier(n_estimators = 100).fit(X,A)
propensity_scores = [x[1] for x in adaboost.predict_proba(X)]

In [9]:
propensity_scores = np.array(propensity_scores).reshape(-1,1)
dist_matrix_propensity_score = pairwise_distances(propensity_scores,metric='l1')

### 3. Linear Propensity Score

In [ ]:
#not sure what the logits are for boosting